#### Import Libraries

In [ ]:
import os
import pandas as pd

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table_experiments as dte
from dash.dependencies import Input, Output, State

import plotly.graph_objs as go
from plotly import tools

#### Set Log File

In [ ]:
LOGFILE = 'C:/Users/596596/Documents/GitHub/UVA-DataPalooza/data/predictions_for_plotly.csv'
DATA = pd.read_csv(LOGFILE)
dataframes = {'DATA': DATA}

#### Create Plotly Dash Application & Server

In [ ]:
app = dash.Dash("UVA DataPalooza Plotly Dash Appication", static_folder='assets')
server = app.server

#### Create Data Object Method

In [ ]:
def get_data_object(user_selection):
    """
    For user selections, return the relevant in-memory data frame.
    """
    return dataframes[user_selection]

#### Create Application Layout

In [ ]:
app.layout = html.Div([
    html.Div([
        
        html.Div([
            html.Div([
                html.Img(src='/assets/uva-datapalooza-2018.png', style={'width':'100%', 'height':'100%'})
            ], className='six columns'),
            
            html.Div([
                html.H2('Plotly Dash Demonstration'),
                html.H2('Stack Over Flow Questions Analysis')
            ], className='five columns offset-by-one'),
            
        ], className='row'),
        
        html.Br([]),
        
        dcc.Dropdown(
            id='field-dropdown',
            options=[{'label': df, 'value': df} for df in dataframes],
            value='DATA',
            clearable=False),
        
        html.Br([]),
        
        dte.DataTable(
            rows=[{}],
            row_selectable=True,
            filterable=True,
            sortable=True,
            selected_row_indices=[],
            id='table'),
        
        html.Br([]),
        
        dcc.Graph(id='dot-graph')
        
    ], className='eleven columns')

], className='container')

#### Create Application Callbacks

In [ ]:
@app.callback(Output('table', 'rows'), [Input('field-dropdown', 'value')])
def update_table(user_selection):
    df = get_data_object(user_selection)
    return df.to_dict('records')


@app.callback(Output('dot-graph', 'figure'), [Input('field-dropdown', 'value')])
def update_dot_graph(user_selection):
    df = get_data_object(user_selection)

    trace1 = go.Scatter(
        y = df.Actual[:10].unique(), 
        x = df.index[:10], 
        marker = dict(color = "orange", size = 12), 
        mode = "markers", 
        name = "Actual")

    trace2 = go.Scatter(
        y = df.Predictions[:10].unique(), 
        x = df.index[:10], 
        marker = dict(color = "blue", size = 12), 
        mode = "markers", 
        name = "Predictions")
    
    return {
        'data': [trace1, trace2],
        'layout': {'title': 'Stack Overflow Questions',
                   'xaxis': {'title': 'Actual'},
                   'yaxis': {'title': 'Predicted'}
                  }
    }

#### Run Application

In [ ]:
external_css = [
    "https://cdnjs.cloudflare.com/ajax/libs/normalize/7.0.0/normalize.min.css",
    "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
    "https://fonts.googleapis.com/css?family=Open+Sans|Roboto",
    "https://codepen.io/mrushin/pen/bQVEgg.css",
]

for css in external_css:
    app.css.append_css({"external_url": css})

if __name__ == '__main__':
    app.run_server(debug=False)